In [1]:
# Data Dictionary for Male and Female as per the excel sheet. 
# This dictionary will be pushed to MongoDb in future and then will be called from there. 

def cds_cal(gender,age,lifestyle,meals, hei_file):
    
    '''
    DESCRIPTION
    ==============
    
    This is the main module for Composite Dish Score (CDS).
    It has sub-functions written inside this module for calculation of Ideal Macros and actual Macros present in Aramark dishes.
    
    About Data Dictionary: 
    This is the dictionary of Ideal calories intake requirement calculated from the USDA for Males and Females as per their age and lifestyle. 
    
    About Sub-Functions:   
    1) get_ideal_daily_macros(): Based on the Data Dictionary, ideal macros are being calculated in this function.
    Ideal Macros that are being considered are: Protein, Carbohydrates, Saturated Fat, Total Fat, Total Sugar, Dietary Fibre.
    It will calculate ideal breakfast, ideal lunch and ideal dinner in the ratio if 1:2:2.
    2) cds_cal():  This function calculates the weighted difference b/w ideal and actual macros and the using the weighted score and HEI score it calculates the CDS score.	
    3) Diff_cal(): This function calls the cds_cal function for all the meal periods (Breakfast, Lunch and Dinner) and creates dictionary with Meal period as key and list of Recipes based on their CDS score as values.
     
    List_dict: This is the list of dictionaries created from the Meals API. This will have all the Primary attributes from Aramark data.


    Parameters:
    gender: 'M' or 'F' : Gender for which CDS has to be calculated.
    age: '1-3','4-8'... : Age Range as per the data dictionary.
    meals: Meals API from Backend.
    hei_file: HEI_score calculation file from backend.
    

    '''
    data = {
            'M': {
                '1-3': {
                    'Sedentary': '1000',
                    'Moderate': '1200',
                    'Active': '1200',
                    'Fiber': '14.0'
                    },
                '4-8': {
                    'Sedentary': '1320',
                    'Moderate': '1520',
                    'Active': '1760',
                    'Fiber': '19.6'
                    },
                '9-13': {
                    'Sedentary': '1760',
                    'Moderate': '2000',
                    'Active': '2280',
                    'Fiber': '25.2'
                    },
                '14-18': {
                    'Sedentary': '2280',
                    'Moderate': '2680',
                    'Active': '3080',
                    'Fiber': '30.8'
                    },
                '19-30': {
                    'Sedentary': '2466',
                    'Moderate': '2733',
                    'Active': '3000',
                    'Fiber': '33.6'
                    },
                '31-50': {
                    'Sedentary': '2300',
                    'Moderate': '2550',
                    'Active': '2850',
                    'Fiber': '30.8'
                    },
                '51-99': {
                    'Sedentary': '2066',
                    'Moderate': '2300',
                    'Active': '2600',
                    'Fiber': '28.0'
                    }
                },
        'F': {
                '1-3': {
                    'Sedentary': '1000',
                    'Moderate': '1100',
                    'Active': '1200',
                    'Fiber': '14.0'
                    },
                '4-8': {
                    'Sedentary': '1240',
                    'Moderate': '1480',
                    'Active': '1640',
                    'Fiber': '16.8'
                    },
                '9-13': {
                    'Sedentary': '1520',
                    'Moderate': '1840',
                    'Active': '2040',
                    'Fiber': '22.4'
                    },
                '14-18': {
                    'Sedentary': '1800',
                    'Moderate': '2000',
                    'Active': '2400',
                    'Fiber': '25.2'
                    },
                '19-30': {
                    'Sedentary': '1933',
                    'Moderate': '2133',
                    'Active': '2400',
                    'Fiber': '28.0'
                    },
                '31-50': {
                    'Sedentary': '1800',
                    'Moderate': '2000',
                    'Active': '2200',
                    'Fiber': '25.2'
                    },
                '51-99': {
                    'Sedentary': '1600',
                    'Moderate': '1800',
                    'Active': '2066',
                    'Fiber': '22.4'
                    }
                }
            }          

    # Function to calcuate the ideal Macros based on Age, Gender and Lifestyle for a day.

    def get_ideal_daily_macros(gender,age,lifestyle):
        
        '''
        Based on the Data Dictionary, ideal macros are being calculated in this function.
        Ideal Macros that are being considered are:
        Protein, Carbohydrates, Saturated Fat, Total Fat, Total Sugar, Dietary Fibre.
        It will calculate ideal breakfast, ideal lunch and ideal dinner in the ratio if 1:2:2.

        '''
        # Call the get_cal function to get the calories for the given gender, age and lifestyle
        cals=int(data[gender][age][lifestyle])

        # As per standards protein intake should be minimum of 10 percent of dailay calorie intake
        cal_in_protein = cals * 0.1
        # As per standards, 1 gram of protein contains 4 calories
        protein_in_grams = cal_in_protein / 4

        # As per standards carb intake should be minimum of 45 percent of dailay calorie intake
        cal_in_carbs = cals * 0.45
        # As per standards, 1 gram of carb contains 4 calories
        carbs_in_grams = cal_in_carbs / 4

        # As per standards saturated fat intake should be minimum of 5 percent of dailay calorie intake
        cal_in_sat_fat = cals * 0.05
        # As per standards, 1 gram of saturated contains 9 calories
        sat_fat_in_grams = cal_in_sat_fat / 9

        # As per standards added sugar intake should be minimum of 5 percent of dailay calorie intake
        cal_in_sugar = cals * 0.05
        # As per standards, 1 gram of added sugar contains 4 calories
        sugar_in_grams = cal_in_sugar / 4

        # Total fat is different for different age groups. 
        # As per standards, 1 gram of total fat contains 9 calories
        if age == '1-3':
            cal_in_tfat = cals * 0.3
            tfat_in_grams = cal_in_tfat / 9
        if age in ['4-8','9-13','14-18']:
            cal_in_tfat = cals * 0.25
            tfat_in_grams = cal_in_tfat / 9
        if age in ['19-30','31-50','51-99']:
            cal_in_tfat = cals * 0.2
            tfat_in_grams = cal_in_tfat / 9

        # calcluate fiber based on gender and age group.    
        fiber_in_grams =  float(data[gender][age]['Fiber'])
        
        # Calculate Ideal breakfast, Lunch and Dinner in 1:2:2 ratio.
        
        ideal_bf={}
        ideal_bf['protein'] = protein_in_grams / 5
        ideal_bf['carbohydrate'] = carbs_in_grams / 5
        ideal_bf['saturated_fat'] = sat_fat_in_grams / 5
        ideal_bf['total_sugars'] = sugar_in_grams / 5
        ideal_bf['fiber'] = fiber_in_grams / 5
        ideal_bf['total_fat'] = tfat_in_grams / 5
        
        # Using one dictionary for Lunch and Dinner as the ratio is same. 
        
        ideal_lunch_dinner={}
        ideal_lunch_dinner['protein'] = (protein_in_grams / 5)*2
        ideal_lunch_dinner['carbohydrate'] = (carbs_in_grams / 5)*2
        ideal_lunch_dinner['saturated_fat'] = (sat_fat_in_grams / 5)*2
        ideal_lunch_dinner['total_sugars'] = (sugar_in_grams / 5)*2
        ideal_lunch_dinner['fiber'] = (fiber_in_grams / 5)*2
        ideal_lunch_dinner['total_fat'] = (tfat_in_grams / 5)*2

        return [ideal_bf,ideal_lunch_dinner]

    # List_dict : This is the list of dictionaries created from the Meals API. 
    # This will have all the Primary attributes from Aramark data.

    list_dict=[]
    for recipe in meals['results']:
        dict1 = {}
        for i in recipe['primary_attributes']:
            if recipe['primary_attributes'][i] == None:
                recipe['primary_attributes'][i] = 0           
        dict1['recipeid'] = recipe['recipe_id']
        dict1['recipe_name'] = recipe['recipe_name']
        dict1['serving_size_number'] = int(1 if recipe['serving_size_number'] is None else recipe['serving_size_number'])
        dict1['meal_period'] = recipe['meal_period_id']
        dict1['protein'] = (float(0 if recipe['primary_attributes']['protein'] is None else recipe['primary_attributes']['protein']))/dict1['serving_size_number']
        dict1['carbohydrate'] = (float(0 if recipe['primary_attributes']['carbohydrate'] is None else recipe['primary_attributes']['carbohydrate']))/dict1['serving_size_number']
        dict1['fiber'] = (float(0 if recipe['primary_attributes']['fiber'] is None else recipe['primary_attributes']['fiber']))/dict1['serving_size_number']
        dict1['total_fat'] = (float(0 if recipe['primary_attributes']['total_fat'] is None else recipe['primary_attributes']['total_fat']))/dict1['serving_size_number']
        dict1['saturated_fat'] = (float(0 if recipe['primary_attributes']['saturated_fat'] is None else recipe['primary_attributes']['saturated_fat']))/dict1['serving_size_number']
        dict1['total_sugars'] = (float(0 if recipe['primary_attributes']['total_sugars'] is None else recipe['primary_attributes']['total_sugars']))/dict1['serving_size_number']
        dict1['marketing_description'] = recipe['marketing_description']
        dict1['allergen_attributes'] = recipe['allergen_attributes']
        dict1['dietary_attributes'] = recipe['dietary_attributes']
        dict1['recipe_marketing_name'] = recipe['recipe_marketing_name']
        dict1['featured'] = recipe['featured']
        dict1['new'] = recipe['new']
        list_dict.append(dict1)
    

    def cds_cal(list_dict,ideal_data,hei_file):
    
        '''
        This function calculates the weighted difference b/w ideal and actual macros 
        and the using the weighted score and HEI score it calculates the CDS score.
        Macronutrient Weighed Score:
        Protein: 3
        Carb: 2
        Total Fats and Saturated Fats: 2
        Sugar: 1
        Dietary Fiber: 1
        HEI: 1
        '''
        cds_dict={}
        hei = 0 
        protein = ((list_dict['protein']- ideal_data['protein'])/ideal_data['protein'])*3
        carbohydrate = ((list_dict['carbohydrate']- ideal_data['carbohydrate'])/ideal_data['carbohydrate'])*2
        fat = (((list_dict['saturated_fat']- ideal_data['saturated_fat'])/ideal_data['saturated_fat']) + ((list_dict['total_fat']- ideal_data['total_fat'])/ideal_data['total_fat'])) * 2 
        sugar = ((list_dict['total_sugars']- ideal_data['total_sugars'])/ideal_data['total_sugars'])*1
        fiber = ((list_dict['fiber']- ideal_data['fiber'])/ideal_data['fiber'])*1
        for j in range(len(hei_file)):
            if list_dict['recipeid'] == hei_file[j]['recipe_id']:
                hei = round(float(hei_file[j]['norm_HEI_score']),3)
                break
        cds_score = protein + carbohydrate + fat + sugar + fiber + hei
        cds_dict['recipe_id'] = list_dict['recipeid']
        cds_dict['recipe_name'] = list_dict['recipe_name']
        cds_dict['cds'] = round(cds_score,2)
        cds_dict['marketing_description'] = list_dict['marketing_description']
        cds_dict['allergen_attributes'] = list_dict['allergen_attributes']
        cds_dict['dietary_attributes'] = list_dict['dietary_attributes']
        cds_dict['featured'] = list_dict['featured']
        cds_dict['new'] = list_dict['new']
#         cds_dict['meal_period'] = list_dict['meal_period']
        return cds_dict


    def diff_cal(gender,age,lifestyle):
        '''
         This function calls the cds_cal function for all the meal periods (Breakfast, Lunch and Dinner) 
         and creates dictionary with Meal period as key and list of Recipes based on their CDS score as values.
        '''
        final_dict = {}
        bf_list = []
        lunch_list = []
        dinner_list = []
        for i in range(len(list_dict)):
            if (1 in list_dict[i]['meal_period']):
                ideal_bf = get_ideal_daily_macros(gender,age,lifestyle)[0]
                cds_dict = cds_cal(list_dict[i],ideal_bf,hei_file)
#                 cds_dict['meal_period'] = 1
                bf_list.append(cds_dict)
            if (2 in list_dict[i]['meal_period']):    
                ideal_lunch_dinner = get_ideal_daily_macros(gender,age,lifestyle)[1]
                cds_dict = cds_cal(list_dict[i],ideal_lunch_dinner,hei_file)
#                 cds_dict['meal_period'] = 2
                lunch_list.append(cds_dict)
            if (3 in list_dict[i]['meal_period']):    
                ideal_lunch_dinner = get_ideal_daily_macros(gender,age,lifestyle)[1]
                cds_dict = cds_cal(list_dict[i],ideal_lunch_dinner,hei_file)
#                 cds_dict['meal_period'] = 3
                dinner_list.append(cds_dict)
        final_dict['1'] = sorted(bf_list,reverse=True, key=lambda k: k['cds'])
        final_dict['2'] = sorted(lunch_list,reverse=True, key=lambda k: k['cds'])
        final_dict['3'] = sorted(dinner_list,reverse=True, key=lambda k: k['cds'])
        return final_dict
#         return sorted(sorted(cds_list,reverse=True, key=lambda k: k['cds']), reverse = True, key = lambda k:k['meal_period'])
    return diff_cal(gender,age,lifestyle)


In [12]:
# Data Dictionary for Male and Female as per the excel sheet. 
# This dictionary will be pushed to MongoDb in future and then will be called from there. 

def cds_cal1(gender,age,lifestyle,meals, hei_file):
    
    '''
    DESCRIPTION
    ==============
    
    This is the main module for Composite Dish Score (CDS).
    It has sub-functions written inside this module for calculation of Ideal Macros and actual Macros present in Aramark dishes.
    
    About Data Dictionary: 
    This is the dictionary of Ideal calories intake requirement calculated from the USDA for Males and Females as per their age and lifestyle. 
    
    About Sub-Functions:   
    1) get_ideal_daily_macros(): Based on the Data Dictionary, ideal macros are being calculated in this function.
    Ideal Macros that are being considered are: Protein, Carbohydrates, Saturated Fat, Total Fat, Total Sugar, Dietary Fibre.
    It will calculate ideal breakfast, ideal lunch and ideal dinner in the ratio if 1:2:2.
    2) cds_cal():  This function calculates the weighted difference b/w ideal and actual macros and the using the weighted score and HEI score it calculates the CDS score.	
    3) Diff_cal(): This function calls the cds_cal function for all the meal periods (Breakfast, Lunch and Dinner) and creates dictionary with Meal period as key and list of Recipes based on their CDS score as values.
     
    List_dict: This is the list of dictionaries created from the Meals API. This will have all the Primary attributes from Aramark data.


    Parameters:
    gender: 'M' or 'F' : Gender for which CDS has to be calculated.
    age: '1-3','4-8'... : Age Range as per the data dictionary.
    meals: Meals API from Backend.
    hei_file: HEI_score calculation file from backend.
    

    '''
    data = {
            'M': {
                '1-3': {
                    'Sedentary': '1000',
                    'Moderate': '1200',
                    'Active': '1200',
                    'Fiber': '14.0'
                    },
                '4-8': {
                    'Sedentary': '1320',
                    'Moderate': '1520',
                    'Active': '1760',
                    'Fiber': '19.6'
                    },
                '9-13': {
                    'Sedentary': '1760',
                    'Moderate': '2000',
                    'Active': '2280',
                    'Fiber': '25.2'
                    },
                '14-18': {
                    'Sedentary': '2280',
                    'Moderate': '2680',
                    'Active': '3080',
                    'Fiber': '30.8'
                    },
                '19-30': {
                    'Sedentary': '2466',
                    'Moderate': '2733',
                    'Active': '3000',
                    'Fiber': '33.6'
                    },
                '31-50': {
                    'Sedentary': '2300',
                    'Moderate': '2550',
                    'Active': '2850',
                    'Fiber': '30.8'
                    },
                '51-99': {
                    'Sedentary': '2066',
                    'Moderate': '2300',
                    'Active': '2600',
                    'Fiber': '28.0'
                    }
                },
        'F': {
                '1-3': {
                    'Sedentary': '1000',
                    'Moderate': '1100',
                    'Active': '1200',
                    'Fiber': '14.0'
                    },
                '4-8': {
                    'Sedentary': '1240',
                    'Moderate': '1480',
                    'Active': '1640',
                    'Fiber': '16.8'
                    },
                '9-13': {
                    'Sedentary': '1520',
                    'Moderate': '1840',
                    'Active': '2040',
                    'Fiber': '22.4'
                    },
                '14-18': {
                    'Sedentary': '1800',
                    'Moderate': '2000',
                    'Active': '2400',
                    'Fiber': '25.2'
                    },
                '19-30': {
                    'Sedentary': '1933',
                    'Moderate': '2133',
                    'Active': '2400',
                    'Fiber': '28.0'
                    },
                '31-50': {
                    'Sedentary': '1800',
                    'Moderate': '2000',
                    'Active': '2200',
                    'Fiber': '25.2'
                    },
                '51-99': {
                    'Sedentary': '1600',
                    'Moderate': '1800',
                    'Active': '2066',
                    'Fiber': '22.4'
                    }
                }
            }          

    # Function to calcuate the ideal Macros based on Age, Gender and Lifestyle for a day.

    def get_ideal_daily_macros1(gender,age,lifestyle):
        
        '''
        Based on the Data Dictionary, ideal macros are being calculated in this function.
        Ideal Macros that are being considered are:
        Protein, Carbohydrates, Saturated Fat, Total Fat, Total Sugar, Dietary Fibre.
        It will calculate ideal breakfast, ideal lunch and ideal dinner in the ratio if 1:2:2.

        '''
        # Call the get_cal function to get the calories for the given gender, age and lifestyle
        cals=int(data[gender][age][lifestyle])

        # As per standards protein intake should be minimum of 10 percent of dailay calorie intake
        cal_in_protein = cals * 0.1
        # As per standards, 1 gram of protein contains 4 calories
        protein_in_grams = cal_in_protein / 4

        # As per standards carb intake should be minimum of 45 percent of dailay calorie intake
        cal_in_carbs = cals * 0.45
        # As per standards, 1 gram of carb contains 4 calories
        carbs_in_grams = cal_in_carbs / 4

        # As per standards saturated fat intake should be minimum of 5 percent of dailay calorie intake
        cal_in_sat_fat = cals * 0.05
        # As per standards, 1 gram of saturated contains 9 calories
        sat_fat_in_grams = cal_in_sat_fat / 9

        # As per standards added sugar intake should be minimum of 5 percent of dailay calorie intake
        cal_in_sugar = cals * 0.05
        # As per standards, 1 gram of added sugar contains 4 calories
        sugar_in_grams = cal_in_sugar / 4

        # Total fat is different for different age groups. 
        # As per standards, 1 gram of total fat contains 9 calories
        if age == '1-3':
            cal_in_tfat = cals * 0.3
            tfat_in_grams = cal_in_tfat / 9
        if age in ['4-8','9-13','14-18']:
            cal_in_tfat = cals * 0.25
            tfat_in_grams = cal_in_tfat / 9
        if age in ['19-30','31-50','51-99']:
            cal_in_tfat = cals * 0.2
            tfat_in_grams = cal_in_tfat / 9

        # calcluate fiber based on gender and age group.    
        fiber_in_grams =  float(data[gender][age]['Fiber'])
        
        # Calculate Ideal breakfast, Lunch and Dinner in 1:2:2 ratio.
        
        ideal_bf={}
        ideal_bf['protein'] = protein_in_grams / 5
        ideal_bf['carbohydrate'] = carbs_in_grams / 5
        ideal_bf['saturated_fat'] = sat_fat_in_grams / 5
        ideal_bf['total_sugars'] = sugar_in_grams / 5
        ideal_bf['fiber'] = fiber_in_grams / 5
        ideal_bf['total_fat'] = tfat_in_grams / 5
        
        # Using one dictionary for Lunch and Dinner as the ratio is same. 
        
        ideal_lunch_dinner={}
        ideal_lunch_dinner['protein'] = (protein_in_grams / 5)*2
        ideal_lunch_dinner['carbohydrate'] = (carbs_in_grams / 5)*2
        ideal_lunch_dinner['saturated_fat'] = (sat_fat_in_grams / 5)*2
        ideal_lunch_dinner['total_sugars'] = (sugar_in_grams / 5)*2
        ideal_lunch_dinner['fiber'] = (fiber_in_grams / 5)*2
        ideal_lunch_dinner['total_fat'] = (tfat_in_grams / 5)*2

        return [ideal_bf,ideal_lunch_dinner]

    # List_dict : This is the list of dictionaries created from the Meals API. 
    # This will have all the Primary attributes from Aramark data.

    list_dict=[]
    for recipe in meals['results']:
        dict1 = {}
        for i in recipe['primary_attributes']:
            if recipe['primary_attributes'][i] == None:
                recipe['primary_attributes'][i] = 0           
        dict1['recipeid'] = recipe['recipe_id']
        dict1['recipe_name'] = recipe['recipe_name']
        dict1['serving_size_number'] = int( {True: 1, False: recipe['serving_size_number']}  [recipe['serving_size_number'] == None ]  )
        dict1['meal_period'] = recipe['meal_period_id']
        dict1['protein'] = (float( {True:0, False: recipe['primary_attributes']['protein']} [ recipe['primary_attributes']['protein'] == None ] ))/dict1['serving_size_number']
        dict1['carbohydrate'] = (float({True:0, False: recipe['primary_attributes']['carbohydrate']} [ recipe['primary_attributes']['carbohydrate'] == None] ))/dict1['serving_size_number']
        dict1['fiber'] = (float({True:0, False: recipe['primary_attributes']['fiber']} [ recipe['primary_attributes']['carbohydrate'] == None] ))/dict1['serving_size_number']
        dict1['total_fat'] = (float({True:0, False: recipe['primary_attributes']['total_fat']} [ recipe['primary_attributes']['carbohydrate'] == None] ))/dict1['serving_size_number']
        dict1['saturated_fat'] = (float({True:0, False: recipe['primary_attributes']['saturated_fat']} [ recipe['primary_attributes']['carbohydrate'] == None] ))/dict1['serving_size_number']
        dict1['total_sugars'] = (float({True:0, False: recipe['primary_attributes']['total_sugars']} [ recipe['primary_attributes']['carbohydrate'] == None] ))/dict1['serving_size_number']
        dict1['marketing_description'] = recipe['marketing_description']
        dict1['allergen_attributes'] = recipe['allergen_attributes']
        dict1['dietary_attributes'] = recipe['dietary_attributes']
        dict1['recipe_marketing_name'] = recipe['recipe_marketing_name']
        dict1['featured'] = recipe['featured']
        dict1['new'] = recipe['new']
        list_dict.append(dict1)
    

    def cds_cal1(list_dict,ideal_data,hei_file):
    
        '''
        This function calculates the weighted difference b/w ideal and actual macros 
        and the using the weighted score and HEI score it calculates the CDS score.
        Macronutrient Weighed Score:
        Protein: 3
        Carb: 2
        Total Fats and Saturated Fats: 2
        Sugar: 1
        Dietary Fiber: 1
        HEI: 1
        '''
        cds_dict={}
        hei = 0 
        protein = ((list_dict['protein']- ideal_data['protein'])/ideal_data['protein'])*3
        carbohydrate = ((list_dict['carbohydrate']- ideal_data['carbohydrate'])/ideal_data['carbohydrate'])*2
        fat = (((list_dict['saturated_fat']- ideal_data['saturated_fat'])/ideal_data['saturated_fat']) + ((list_dict['total_fat']- ideal_data['total_fat'])/ideal_data['total_fat'])) * 2 
        sugar = ((list_dict['total_sugars']- ideal_data['total_sugars'])/ideal_data['total_sugars'])*1
        fiber = ((list_dict['fiber']- ideal_data['fiber'])/ideal_data['fiber'])*1
        for j in range(len(hei_file)):
            if list_dict['recipeid'] == hei_file[j]['recipe_id']:
                hei = round(float(hei_file[j]['norm_HEI_score']),3)
                break
        cds_score = protein + carbohydrate + fat + sugar + fiber + hei
        cds_dict['recipe_id'] = list_dict['recipeid']
        cds_dict['recipe_name'] = list_dict['recipe_name']
        cds_dict['cds'] = round(cds_score,2)
        cds_dict['marketing_description'] = list_dict['marketing_description']
        cds_dict['allergen_attributes'] = list_dict['allergen_attributes']
        cds_dict['dietary_attributes'] = list_dict['dietary_attributes']
        cds_dict['featured'] = list_dict['featured']
        cds_dict['new'] = list_dict['new']
#         cds_dict['meal_period'] = list_dict['meal_period']
        return cds_dict


    def diff_cal1(gender,age,lifestyle):
        '''
         This function calls the cds_cal function for all the meal periods (Breakfast, Lunch and Dinner) 
         and creates dictionary with Meal period as key and list of Recipes based on their CDS score as values.
        '''
        final_dict = {}
        bf_list = []
        lunch_list = []
        dinner_list = []
        for i in range(len(list_dict)):
            if (1 in list_dict[i]['meal_period']):
                ideal_bf = get_ideal_daily_macros1(gender,age,lifestyle)[0]
                cds_dict = cds_cal1(list_dict[i],ideal_bf,hei_file)
#                 cds_dict['meal_period'] = 1
                bf_list.append(cds_dict)
            if (2 in list_dict[i]['meal_period']):    
                ideal_lunch_dinner = get_ideal_daily_macros1(gender,age,lifestyle)[1]
                cds_dict = cds_cal1(list_dict[i],ideal_lunch_dinner,hei_file)
#                 cds_dict['meal_period'] = 2
                lunch_list.append(cds_dict)
            if (3 in list_dict[i]['meal_period']):    
                ideal_lunch_dinner = get_ideal_daily_macros1(gender,age,lifestyle)[1]
                cds_dict = cds_cal1(list_dict[i],ideal_lunch_dinner,hei_file)
#                 cds_dict['meal_period'] = 3
                dinner_list.append(cds_dict)
        final_dict['1'] = sorted(bf_list,reverse=True, key=lambda k: k['cds'])
        final_dict['2'] = sorted(lunch_list,reverse=True, key=lambda k: k['cds'])
        final_dict['3'] = sorted(dinner_list,reverse=True, key=lambda k: k['cds'])
        return final_dict
#         return sorted(sorted(cds_list,reverse=True, key=lambda k: k['cds']), reverse = True, key = lambda k:k['meal_period'])
    return diff_cal1(gender,age,lifestyle)


## Testing

In [8]:
import json
import requests

with open(r"C:\Users\varun\Downloads\HEI Script\HEI Script\data\Updated_0-1.json") as file:
    hei_file = json.load(file)
    
url = "http://flask-test-aramark.azurewebsites.net/admin/api/meals/"
 
response = requests.get(url)


meals = response.json()

In [17]:
import timeit
starttime = timeit.default_timer()
a = cds_cal('M','19-30','Moderate',meals,hei_file)
print("The time difference is :", timeit.default_timer() - starttime)

The time difference is : 0.0028159999999388674


In [18]:
starttime = timeit.default_timer()
b = cds_cal1('M','19-30','Moderate',meals,hei_file)
print("The time difference is :", timeit.default_timer() - starttime)

The time difference is : 0.0028732000000672997
